In [1]:
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch

# データ読み込み
iris = datasets.load_iris()
data = iris['data']
target = iris['target']

# 学習データと検証データに分割
x_train, x_valid, y_train, y_valid = train_test_split(data, target, shuffle=True)

# 特徴量の標準化
scaler = StandardScaler()
scaler.fit(x_train)

x_train = scaler.transform(x_train)
x_valid = scaler.transform(x_valid)

# Tensor型に変換
# 学習に入れるときはfloat型 or long型になっている必要があるのここで変換してしまう
x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).long()
x_valid = torch.from_numpy(x_valid).float()
y_valid = torch.from_numpy(y_valid).long()

print('x_train : ', x_train.shape)
print('y_train : ', y_train.shape)
print('x_valid : ', x_valid.shape)
print('y_valid : ', y_valid.shape)

x_train :  torch.Size([112, 4])
y_train :  torch.Size([112])
x_valid :  torch.Size([38, 4])
y_valid :  torch.Size([38])


In [2]:
from torch.utils.data import TensorDataset

train_dataset = TensorDataset(x_train, y_train)
valid_dataset = TensorDataset(x_valid, y_valid)

# 動作確認
# indexを指定すればデータを取り出すことができます。
index = 0
print(train_dataset.__getitem__(index)[0].size())
print(train_dataset.__getitem__(index)[1])

torch.Size([4])
tensor(0)


In [3]:
from torch.utils.data import DataLoader

batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

# 動作確認
# こんな感じでバッチ単位で取り出す子ができます。
# イテレータに変換
batch_iterator = iter(train_dataloader)
# 1番目の要素を取り出す
inputs, labels = next(batch_iterator)
print(inputs.size())
print(labels.size())

torch.Size([32, 4])
torch.Size([32])


In [4]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4, 50)
        self.fc2 = nn.Linear(50, 3)
    
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return x

net = Net()
print(net)

Net(
  (fc1): Linear(in_features=4, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=3, bias=True)
)


In [5]:
import torch.nn as nn

criterion = nn.CrossEntropyLoss()

In [6]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)

In [7]:
# エポック数
num_epochs = 50

# 学習時と検証時で分けるためディクショナリを用意
dataloaders_dict = {
    'train': train_dataloader,
    'val': valid_dataloader
}

for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch+1, num_epochs))
    print('-------------')
    
    for phase in ['train', 'val']:
        
        if phase == 'train':
            # モデルを訓練モードに設定
            net.train()
        else:
            # モデルを推論モードに設定
            net.eval()
        
        # 損失和
        epoch_loss = 0.0
        # 正解数
        epoch_corrects = 0
        
        # DataLoaderからデータをバッチごとに取り出す
        for inputs, labels in dataloaders_dict[phase]:
            
            # optimizerの初期化
            optimizer.zero_grad()
            
            # 学習時のみ勾配を計算させる設定にする
            with torch.set_grad_enabled(phase == 'train'):
                outputs = net(inputs)
                
                # 損失を計算
                loss = criterion(outputs, labels)
                
                # ラベルを予測
                _, preds = torch.max(outputs, 1)
                
                # 訓練時はバックプロパゲーション
                if phase == 'train':
                    # 逆伝搬の計算
                    loss.backward()
                    # パラメータの更新
                    optimizer.step()
                
                # イテレーション結果の計算
                # lossの合計を更新
                # PyTorchの仕様上各バッチ内での平均のlossが計算される。
                # データ数を掛けることで平均から合計に変換をしている。
                # 損失和は「全データの損失/データ数」で計算されるため、
                # 平均のままだと損失和を求めることができないため。
                epoch_loss += loss.item() * inputs.size(0)
                
                # 正解数の合計を更新
                epoch_corrects += torch.sum(preds == labels.data)

        # epochごとのlossと正解率を表示
        epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
        epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

Epoch 1/50
-------------
train Loss: 1.0838 Acc: 0.5357
val Loss: 1.0781 Acc: 0.6579
Epoch 2/50
-------------
train Loss: 1.0788 Acc: 0.5804
val Loss: 1.0728 Acc: 0.6579
Epoch 3/50
-------------
train Loss: 1.0737 Acc: 0.6250
val Loss: 1.0676 Acc: 0.6842
Epoch 4/50
-------------
train Loss: 1.0685 Acc: 0.6875
val Loss: 1.0626 Acc: 0.6842
Epoch 5/50
-------------
train Loss: 1.0635 Acc: 0.7232
val Loss: 1.0571 Acc: 0.7368
Epoch 6/50
-------------
train Loss: 1.0582 Acc: 0.7500
val Loss: 1.0518 Acc: 0.7368
Epoch 7/50
-------------
train Loss: 1.0529 Acc: 0.7500
val Loss: 1.0464 Acc: 0.7632
Epoch 8/50
-------------
train Loss: 1.0477 Acc: 0.7500
val Loss: 1.0410 Acc: 0.7895
Epoch 9/50
-------------
train Loss: 1.0423 Acc: 0.7589
val Loss: 1.0353 Acc: 0.7895
Epoch 10/50
-------------
train Loss: 1.0368 Acc: 0.7589
val Loss: 1.0300 Acc: 0.8158
Epoch 11/50
-------------
train Loss: 1.0315 Acc: 0.7679
val Loss: 1.0245 Acc: 0.8158
Epoch 12/50
-------------
train Loss: 1.0261 Acc: 0.7589
val Lo

In [8]:
# 予測用のダミーデータ
x = torch.randn([3, 4])
preds = net(x)